### Importing Libraries

In [3]:
from collections import deque, namedtuple
import random
import nntplib as nn
from typing import Deque
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import pandas as pd
from matplotlib import pyplot as plt

from parameters_global import *


### GlobalParams

Dieser Code definiert verschiedene Konstanten und Parameter, die in einem Handelsalgorithmus verwendet werden, der auf einem verstärkten Lernen basiert. 

1. STATE_SPACE: Dies ist die Größe des Zustandsraums, der die Merkmale enthält, die der Agent betrachtet, um seine Entscheidungen zu treffen. In diesem Fall beträgt der Zustandsraum 34.

2. ACTION_SPACE: Dies ist die Größe des Aktionsraums, der die möglichen Aktionen enthält, die der Agent ausführen kann. Hier hat der Aktionsraum eine Größe von 3, da der Agent zwischen den Aktionen "Verkauf" (-1), "Nichtstun" (0) und "Kauf" (1) wählen kann.

3. ACTION_LOW und ACTION_HIGH: Dies sind die Grenzen des Aktionsraums, die den niedrigsten und höchsten Wert der Aktionen angeben, die der Agent ausführen kann.

4. GAMMA: Dies ist der Diskontierungsfaktor, der angibt, wie stark der Agent zukünftige Belohnungen gewichtet. Ein hoher GAMMA-Wert (nahe 1) bedeutet, dass der Agent langfristige Belohnungen stärker berücksichtigt.

5. TAU: Dies ist der Wert für die Soft-Update-Methode, mit der die Zielnetzwerkparameter allmählich an die aktuellen Netzwerkparameter angepasst werden, um eine stabilere Lernleistung zu erzielen.

6. EPS_START, EPS_END und EPS_DECAY: Diese Parameter steuern die Exploration des Agenten während des Lernprozesses. Der Agent verwendet eine epsilon-greedy Policy, um zwischen Exploration (zufällige Aktionen) und Exploitation (Aktionen basierend auf dem bisher Gelernten) zu wählen. EPS_START gibt den Startwert für die Wahrscheinlichkeit der Exploration an, während EPS_END den Endwert angibt. EPS_DECAY bestimmt die Geschwindigkeit, mit der die Exploration im Laufe der Zeit reduziert wird.

7. MEMORY_LEN: Dies ist die maximale Größe des Erfahrungsspeichers, in dem der Agent vergangene Erfahrungen speichert, um sie für das Training zu verwenden.

8. MEMORY_THRESH: Die Anzahl der Erfahrungen, die im Erfahrungsspeicher gesammelt werden müssen, bevor das Training des Agenten beginnt.

9. BATCH_SIZE: Die Größe der Stichprobe, die aus dem Erfahrungsspeicher für jedes Training gezogen wird.

10. LR_DQN: Die Lernrate des Deep-Q-Networks, die die Geschwindigkeit des Lernprozesses beeinflusst.

11. LEARN_AFTER: Die Anzahl der Erfahrungen, die im Erfahrungsspeicher gesammelt werden müssen, bevor der Agent mit dem Training beginnt.

12. LEARN_EVERY: Die Häufigkeit, mit der der Agent trainiert wird, nachdem er das erste Mal gelernt hat.

13. UPDATE_EVERY: Die Häufigkeit, mit der die Zielnetzwerkparameter aktualisiert werden.

14. COST: Die Handelskosten, die bei jeder Aktion berücksichtigt werden.

15. CAPITAL: Der Anfangsbetrag des Kapitals, mit dem der Agent handelt.

16. NEG_MUL: Ein Multiplikator, der verwendet wird, um den Agenten risikoscheuer gegenüber einer negativen Rendite zu machen. Ein höherer Wert bedeutet, dass der Agent stärker bestraft wird, wenn er Verluste erleidet.

In [4]:
STATE_SPACE = 34  #geändert anstatt 28
ACTION_SPACE = 3

ACTION_LOW = -1
ACTION_HIGH = 1

GAMMA = 0.9995
TAU = 1e-3
EPS_START = 1.0
EPS_END = 0.1
EPS_DECAY = 0.9

MEMORY_LEN = 10000
MEMORY_THRESH = 500
BATCH_SIZE = 200

LR_DQN = 5e-4

LEARN_AFTER = MEMORY_THRESH
LEARN_EVERY = 3
UPDATE_EVERY = 9

COST = 3e-4
CAPITAL = 100000
NEG_MUL = 2

### Getting the Data

Die Klasse "DataGetter" dient dazu, historische Daten für bestimmte Vermögenswerte zu sammeln und sie für einen Handelsalgorithmus vorzubereiten. Sie kann Daten für einen bestimmten Vermögenswert über einen angegebenen Zeitraum abrufen und berechnet verschiedene Merkmale wie Renditen, Handelsvolumenänderungen, Volatilität (Bollinger-Bands), Moving Average Convergence Divergence (MACD) und den Relative Strength Indicator (RSI). Diese Merkmale sind entscheidend für den Handelsalgorithmus, um fundierte Entscheidungen zu treffen. Die Methode "scaleData" normalisiert die Daten, um sicherzustellen, dass alle Merkmale auf einer ähnlichen Skala liegen. Durch die Nutzung dieser Klasse kann der Handelsalgorithmus auf gut vorbereitete historische Daten zugreifen und somit optimale Handelsentscheidungen treffen.

In [5]:
class DataGetter:
  """
  Klasse um Daten zu Erstellen
  """

  def __init__(self, asset="BTC-USD", start_date=None, end_date=None, freq="1d", 
               timeframes=[1, 2, 5, 10, 20, 40]):
    self.asset = asset
    self.sd = start_date
    self.ed = end_date
    self.freq = freq

    self.timeframes = timeframes
    self.getData()

    self.scaler = StandardScaler()
    self.scaler.fit(self.data[:, 1:])


  def getData(self):
    
    asset = self.asset  
    if self.sd is not None and self.ed is not None:
      df =  yf.download([asset], start=self.sd, end=self.ed, interval=self.freq)
      df_spy = yf.download(["BTC-USD"], start=self.sd, end=self.ed, interval=self.freq)
    elif self.sd is None and self.ed is not None:
      df =  yf.download([asset], end=self.ed, interval=self.freq)
      df_spy = yf.download(["BTC-USD"], end=self.ed, interval=self.freq)
    elif self.sd is not None and self.ed is None:
      df =  yf.download([asset], start=self.sd, interval=self.freq)
      df_spy = yf.download(["BTC-USD"], start=self.sd, interval=self.freq)
    else:
      df = yf.download([asset], period="max", interval=self.freq)
      df_spy = yf.download(["BTC-USD"], interval=self.freq)
    
    df["rf"] = df["Adj Close"].pct_change().shift(-1)

    for i in self.timeframes:
      df_spy[f"spy_ret-{i}"] = df_spy["Adj Close"].pct_change(i)
      df_spy[f"spy_v-{i}"] = df_spy["Volume"].pct_change(i)

      df[f"r-{i}"] = df["Adj Close"].pct_change(i)      
      df[f"v-{i}"] = df["Volume"].pct_change(i)
    
    for i in [5, 10, 20, 40]:
      df[f'sig-{i}'] = np.log(1 + df["r-1"]).rolling(i).std()

    df["macd_lmw"] = df["r-1"].ewm(span=26, adjust=False).mean()
    df["macd_smw"] = df["r-1"].ewm(span=12, adjust=False).mean()
    df["macd_bl"] = df["r-1"].ewm(span=9, adjust=False).mean()
    df["macd"] = df["macd_smw"] - df["macd_lmw"]

    rsi_lb = 5
    pos_gain = df["r-1"].where(df["r-1"] > 0, 0).ewm(rsi_lb).mean()
    neg_gain = df["r-1"].where(df["r-1"] < 0, 0).ewm(rsi_lb).mean()
    rs = np.abs(pos_gain/neg_gain)
    df["rsi"] = 100 * rs/(1 + rs)

    bollinger_lback = 10
    df["bollinger"] = df["r-1"].ewm(bollinger_lback).mean()
    df["low_bollinger"] = df["bollinger"] - 2 * df["r-1"].rolling(bollinger_lback).std()
    df["high_bollinger"] = df["bollinger"] + 2 * df["r-1"].rolling(bollinger_lback).std()
    # print(df.columns)
    # print(df_spy.columns)
    # SP500
    # df = df.merge(df_spy[[f"spy_ret-{i}" for i in self.timeframes] + [f"spy_sig-{i}" for i in [5, 10, 20, 40]]], 
    #               how="left", right_index=True, left_index=True)
    df = df.merge(df_spy[[f"spy_ret-{i}" for i in self.timeframes]], 
              how="left", right_index=True, left_index=True)


    for c in df.columns:
      df[c].interpolate('linear', limit_direction='both', inplace=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    self.frame = df
    self.data = np.array(df.iloc[:, 6:])
    return


  def scaleData(self):
    self.scaled_data = self.scaler.fit_transform(self.data[:, 1:])
    return


  def __len__(self):
    return len(self.data)


  def __getitem__(self, idx, col_idx=None):
    if col_idx is None:
      return self.data[idx]
    elif col_idx < len(list(self.data.columns)):
      return self.data[idx][col_idx]
    else:
      raise IndexError

### AgentMemory

Der gegebene Code enthält die Implementierung einer Replay Memory-Klasse für einen Agenten im Verstärkungslernen. Diese Klasse ermöglicht es dem Agenten, frühere Erfahrungen in Form von Zustandsübergängen zu speichern und für das Training zu nutzen. Ein Zustandsübergang besteht aus den Komponenten "Zustände", "Aktionen", "Belohnungen", "Folgezustände" und "Abschlüsse". Die Replay Memory-Klasse verwendet ein Deque als internen Speicher mit einer festen Kapazität, um die letzten Erfahrungen beizubehalten und ältere Erfahrungen zu verwerfen, wenn die Kapazitätsgrenze erreicht ist. Die Methode "sample" zieht zufällig eine Stichprobe von Erfahrungen aus dem Speicher, um das Lernen zu diversifizieren. Insgesamt ermöglicht die Replay Memory-Klasse dem Agenten, aus vergangenen Erfahrungen zu lernen und seine Entscheidungsfindung zu verbessern, um eine optimale Handelsstrategie zu entwickeln.

In [6]:
Transition = namedtuple("Transition", ["States", "Actions", "Rewards", "NextStates", "Dones"])


class ReplayMemory:
  """
  Implementierung des Agenten
  """
  def __init__(self, capacity=MEMORY_LEN):
    self.memory = Deque(maxlen=capacity)

  def store(self, t):
    self.memory.append(t)

  def sample(self, n):
    a = random.sample(self.memory, n)
    return a

  def __len__(self):
    return len(self.memory)

### Agent

Der gegebene Code implementiert ein Duelling Deep Q Network (DQN) für einen Handelsagenten im Verstärkungslernen. Die Architektur des neuronalen Netzwerks ist in der Klasse `DuellingDQN` definiert, während die Handelsentscheidungen und das Training durch die Klasse `DQNAgent` erfolgen.

Die `DuellingDQN`-Klasse definiert die Struktur des neuronalen Netzwerks mit mehreren vollständig verbundenen Schichten. Es verwendet ReLU-, Tanh- und Sigmoid-Aktivierungsfunktionen und eine Softmax-Aktivierungsfunktion in der letzten Schicht, um Aktionen vorherzusagen.

Der `DQNAgent` verwendet zwei Instanzen des DuellingDQN-Netzwerks: `actor_online` für die Entscheidungsfindung und `actor_target` für die zielgerichtete Aktualisierung. Die Erfahrungen des Agenten werden in einer Replay Memory-Klasse (`memory`) gespeichert und verwendet, um das Netzwerk zu trainieren. Der Agent verwendet die Bellman-Gleichung und den Adam-Optimizer, um den Verlust des neuronalen Netzwerks zu berechnen und zu minimieren. Zusätzlich wird das Target-Netzwerk durch weiche Aktualisierung aktualisiert, um das Lernen zu stabilisieren.

Der Agent kann basierend auf einem Zustand eine Aktion auswählen. Er kann entweder die Aktion mit dem höchsten Q-Wert wählen oder eine zufällige Aktion auswählen, um die Exploration und Ausbeutung auszubalancieren.

In [7]:
class DuellingDQN(nn.Module):
  """
  Architektur
  """

  def __init__(self, input_dim=STATE_SPACE, output_dim=ACTION_SPACE):
    super(DuellingDQN, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim

    self.fc1 = nn.Linear(self.input_dim, 500)
    self.fc2 = nn.Linear(500, 500)
    self.fc3 = nn.Linear(500, 300)
    self.fc4 = nn.Linear(300, 200)
    self.fc5 = nn.Linear(200, 10)

    self.fcs = nn.Linear(10, 1)
    self.fcp = nn.Linear(10, self.output_dim)
    self.fco = nn.Linear(self.output_dim + 1, self.output_dim)

    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()
    self.sig = nn.Sigmoid()
    self.sm = nn.Softmax(dim=1)

  def forward(self, state):
    x = self.relu(self.fc1(state))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    x = self.relu(self.fc4(x))
    x = self.relu(self.fc5(x))
    xs = self.relu(self.fcs(x))
    xp = self.relu(self.fcp(x))

    x = xs + xp - xp.mean()
    return x



class DQNAgent:
  """
  Implementierung
  """
  DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


  def __init__(self, actor_net=DuellingDQN, memory=ReplayMemory()):
    
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.actor_online = actor_net(STATE_SPACE, ACTION_SPACE).to(DEVICE)
    self.actor_target = actor_net(STATE_SPACE, ACTION_SPACE).to(DEVICE)
    self.actor_target.load_state_dict(self.actor_online.state_dict())
    self.actor_target.eval()

    self.memory = memory

    self.actor_criterion = nn.MSELoss()
    self.actor_op = optim.Adam(self.actor_online.parameters(), lr=LR_DQN)

    self.t_step = 0


  def act(self, state, eps=0.):
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    self.t_step += 1
    state = torch.from_numpy(state).float().to(DEVICE).view(1, -1)
    
    self.actor_online.eval()
    with torch.no_grad():
      actions = self.actor_online(state)
    self.actor_online.train()

    if random.random() > eps:
      act = np.argmax(actions.cpu().data.numpy())
    else:
      act = random.choice(np.arange(ACTION_SPACE))
    return int(act)


  def learn(self):
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if len(self.memory) <= MEMORY_THRESH:
      return 0

    if self.t_step > LEARN_AFTER and self.t_step % LEARN_EVERY==0:
      batch = self.memory.sample(BATCH_SIZE)

      states = np.vstack([t.States for t in batch])
      states = torch.from_numpy(states).float().to(DEVICE)

      actions = np.vstack([t.Actions for t in batch])
      actions = torch.from_numpy(actions).float().to(DEVICE)

      rewards = np.vstack([t.Rewards for t in batch])
      rewards = torch.from_numpy(rewards).float().to(DEVICE)

      next_states = np.vstack([t.NextStates for t in batch])
      next_states = torch.from_numpy(next_states).float().to(DEVICE)

      dones = np.vstack([t.Dones for t in batch]).astype(np.uint8)
      dones = torch.from_numpy(dones).float().to(DEVICE)

      # Update
      # Berechnung nächster Schritte
      next_state_values = self.actor_target(next_states).max(1)[0].unsqueeze(1)
      y = rewards + (1-dones) * GAMMA * next_state_values
      state_values = self.actor_online(states).gather(1, actions.type(torch.int64))
      # Berechne Verlust
      actor_loss = self.actor_criterion(y, state_values)
      # Minimiere Verlust
      self.actor_op.zero_grad()
      actor_loss.backward()
      self.actor_op.step()

      if self.t_step % UPDATE_EVERY == 0:
        self.soft_update(self.actor_online, self.actor_target)
      # return actor_loss.item()


  def soft_update(self, local_model, target_model, tau=TAU):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

### Environment

Die gegebene Code-Implementierung enthält eine Handelsumgebung für eine Krypto-Währung im Verstärkungslernen. Die Klasse `SingleAssetTradingEnvironment` repräsentiert diese Umgebung, in der der Agent mit den Funktionen `reset` und `step` interagiert.

Der Konstruktor initialisiert die Handelsumgebung mit verschiedenen Parametern wie historischen Preisdaten, anfänglichem Kapital, Transaktionskostenrate und anderen steuernden Parametern. Die Methode `reset` setzt die Umgebung auf den Anfangszustand zurück, während `step` es dem Agenten ermöglicht, eine Aktion auszuführen und den nächsten Zustand, die Belohnung und den Terminalstatus zu erhalten.

Die Handelsumgebung ermöglicht es dem Agenten, Handelsentscheidungen zu treffen und seine Strategie im Laufe der Zeit zu verbessern. Die Umgebung kann an spezifische Handelsszenarien angepasst werden, indem Parameter wie Kapitalanteil, Schwellenwerte und andere Aspekte der Umgebung geändert werden. Insgesamt bietet die Handelsumgebung eine flexible Plattform, um Handelsstrategien im Kontext von Verstärkungslernen zu erforschen und zu entwickeln.

In [8]:
class SingleAssetTradingEnvironment:
  """
  Trading Umgebung für eine Krypto-Währung
  Der Agent interagiert mit der Umgebungsklasse über die Funktion step().
  Aktionsraum: {-1: Verkaufen, 0: Nichts tun, 1: Kaufen}
  """

  def __init__(self, asset_data,
               initial_money=CAPITAL, trans_cost=COST, store_flag=1, asset_ph=0, 
               capital_frac=0.2, running_thresh=0.1, cap_thresh=0.3):

    self.past_holding = asset_ph
    self.capital_frac = capital_frac # Fraction of capital to invest each time.
    self.cap_thresh = cap_thresh
    self.running_thresh = running_thresh
    self.trans_cost = trans_cost

    self.asset_data = asset_data
    self.terminal_idx = len(self.asset_data) - 1
    self.scaler = self.asset_data.scaler    

    self.initial_cap = initial_money

    self.capital = self.initial_cap
    self.running_capital = self.capital
    self.asset_inv = self.past_holding

    self.pointer = 0
    self.next_return, self.current_state = 0, None
    self.prev_act = 0
    self.current_act = 0
    self.current_reward = 0
    self.current_price = self.asset_data.frame.iloc[self.pointer, :]['Adj Close']
    self.done = False

    self.store_flag = store_flag
    if self.store_flag == 1:
      self.store = {"action_store": [],
                    "reward_store": [],
                    "running_capital": [],
                    "port_ret": []}


  def reset(self):
    self.capital = self.initial_cap
    self.running_capital = self.capital
    self.asset_inv = self.past_holding

    self.pointer = 0
    self.next_return, self.current_state = self.get_state(self.pointer)
    self.prev_act = 0
    self.current_act = 0
    self.current_reward = 0
    self.current_price = self.asset_data.frame.iloc[self.pointer, :]['Adj Close']
    self.done = False
    
    if self.store_flag == 1:
      self.store = {"action_store": [],
                    "reward_store": [],
                    "running_capital": [],
                    "port_ret": []}

    return self.current_state


  def step(self, action):
    self.current_act = action
    self.current_price = self.asset_data.frame.iloc[self.pointer, :]['Adj Close']
    self.current_reward = self.calculate_reward()
    self.prev_act = self.current_act
    self.pointer += 1
    self.next_return, self.current_state = self.get_state(self.pointer)
    self.done = self.check_terminal()

    if self.done:
      reward_offset = 0
      ret = (self.store['running_capital'][-1]/self.store['running_capital'][-0]) - 1
      if self.pointer < self.terminal_idx:
        reward_offset += -1 * max(0.5, 1 - self.pointer/self.terminal_idx)
      if self.store_flag:
        reward_offset += 10 * ret
      self.current_reward += reward_offset

    if self.store_flag:
      self.store["action_store"].append(self.current_act)
      self.store["reward_store"].append(self.current_reward)
      self.store["running_capital"].append(self.capital)
      info = self.store
    else:
      info = None
    
    return self.current_state, self.current_reward, self.done, info


  def calculate_reward(self):
    investment = self.running_capital * self.capital_frac
    reward_offset = 0

    # Kaufen
    if self.current_act == 1: 
      if self.running_capital > self.initial_cap * self.running_thresh:
        self.running_capital -= investment
        asset_units = investment/self.current_price
        self.asset_inv += asset_units
        self.current_price *= (1 - self.trans_cost)

    # Verkaufen
    elif self.current_act == -1:
      if self.asset_inv > 0:
        self.running_capital += self.asset_inv * self.current_price * (1 - self.trans_cost)
        self.asset_inv = 0

    # Nichts machen
    elif self.current_act == 0:
      if self.prev_act == 0:
        reward_offset += -0.1
      pass
    
    # Belohnen
    prev_cap = self.capital
    self.capital = self.running_capital + (self.asset_inv) * self.current_price
    reward = 100*(self.next_return) * self.current_act - np.abs(self.current_act - self.prev_act) * self.trans_cost
    if self.store_flag==1:
      self.store['port_ret'].append((self.capital - prev_cap)/prev_cap)
    
    if reward < 0:
      reward *= NEG_MUL  # Agent risikobereiter machen
    reward += reward_offset

    return reward


  def check_terminal(self):
    if self.pointer == self.terminal_idx:
      return True
    elif self.capital <= self.initial_cap * self.cap_thresh:
      return True
    else:
      return False


  def get_state(self, idx):
    state = self.asset_data[idx][1:]
    state = self.scaler.transform(state.reshape(1, -1))

    state = np.concatenate([state, [[self.capital/self.initial_cap,
                                     self.running_capital/self.capital,
                                     self.asset_inv * self.current_price/self.initial_cap,
                                     self.prev_act]]], axis=-1)
    
    next_ret = self.asset_data[idx][0]
    return next_ret, state

### Actual Training

Der Code initialisiert eine Handelsumgebung für verschiedene Kryptowährungen und trainiert einen Agenten mit dem DQNAgent-Modell mittels Verstärkungslernen. Die Hauptschleife führt das Training über mehrere Episoden durch, während der Agent in jeder Episode mit den Umgebungen interagiert und Handelsentscheidungen trifft. Die Leistung des Agenten wird durch kumulierte Belohnungen und Kapitalveränderungen während des Handels bewertet. Der Agent speichert die besten Gewichte während der Validierung, um die besten Ergebnisse zu behalten. Insgesamt wird eine Verstärkungslernen-basierte Handelsstrategie entwickelt, um den Agenten profitabel in verschiedenen Kryptowährungen handeln zu lassen.

In [12]:
# Umgebung und Agenteninitiierung

## Kryptowährungen
asset_codes = ["ETH-USD", "BNB-USD", "XRP-USD", "SOL-USD", "DOGE-USD", 
               "ADA-USD", "MATIC-USD", "AVAX-USD", "WAVES-USD"]

## Trainings- und Testumgebungen
assets = [DataGetter(a, start_date="2015-01-01", end_date="2021-05-01") for a in asset_codes]
test_assets = [DataGetter(a, start_date="2021-05-01", end_date="2022-05-01", freq="1d") for a in asset_codes]
envs = [SingleAssetTradingEnvironment(a) for a in assets]
test_envs = [SingleAssetTradingEnvironment(a) for a in test_assets]

## Agent
memory = ReplayMemory()
agent = DQNAgent(actor_net=DuellingDQN, memory=memory)


# Hauptschleife
N_EPISODES = 20 # Anzahl an Episoden/Epochen
scores = []
eps = EPS_START
act_dict = {0:-1, 1:1, 2:0}

te_score_min = -np.Inf
for episode in range(1, 1 + N_EPISODES):
  counter = 0
  episode_score = 0
  episode_score2 = 0
  test_score = 0
  test_score2 = 0

  for env in envs:
    score = 0
    state = env.reset()
    state = state.reshape(-1, STATE_SPACE)
    while True:
      actions = agent.act(state, eps)
      action = act_dict[actions]
      next_state, reward, done, _ = env.step(action)
      next_state = next_state.reshape(-1, STATE_SPACE)

      t = Transition(state, actions, reward, next_state, done)
      agent.memory.store(t)
      agent.learn()

      state = next_state
      score += reward
      counter += 1
      if done:
        break

    episode_score += score
    episode_score2 += (env.store['running_capital'][-1] - env.store['running_capital'][0])

  scores.append(episode_score)
  eps = max(EPS_END, EPS_DECAY * eps)

  for i, test_env in enumerate(test_envs):
    state = test_env.reset()
    done = False
    score_te = 0
    scores_te = [score_te]

    while True:
      actions = agent.act(state)
      action = act_dict[actions]
      next_state, reward, done, _ = test_env.step(action)
      next_state = next_state.reshape(-1, STATE_SPACE)
      state= next_state
      score_te += reward
      scores_te.append(score_te)
      if done:
        break

    test_score += score_te
    test_score2 += (test_env.store['running_capital'][-1] - test_env.store['running_capital'][0])
  if test_score > te_score_min:
    te_score_min = test_score
    torch.save(agent.actor_online.state_dict(), "online.pt")
    torch.save(agent.actor_target.state_dict(), "target.pt")

  print(f"Episode: {episode}, Train Score: {episode_score:.5f}, Validation Score: {test_score:.5f}")
  print(f"Episode: {episode}, Train Value: ${episode_score2:.5f}, Validation Value: ${test_score2:.5f}", "\n")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********